In [6]:
import csv
import pandas as pd
import networkx as nx
import math

data = []
end_time = [0, 0]
end_time2 = [0, 0]
wait_time_heikin = 0
G = nx.read_weighted_edgelist("dijkstra.txt", create_using=nx.DiGraph)

pattern1 = pd.read_csv("and_data.csv")
with open("and_data.csv", encoding="utf_8") as f:
    reader = csv.reader(f)
    line = [row for row in reader]

for i in range(len(pattern1)):
    aa1 = line[i+1][0].replace(":", " ").split()
    arrival_time = int(aa1[1])*60+int(aa1[0])*3600
    
    counter_no, mi = 0, end_time[0]
    for j, e in enumerate(end_time[1:], 1):
        if mi > e:
            counter_no, mi = j, e
    if counter_no == 0:
        bbb = "X"
    else:
        bbb = "S"

    if len(str(line[i+1][2])) == 0:
        start_time = arrival_time if arrival_time > end_time[counter_no] else end_time[counter_no]
        end_time[counter_no] = start_time
        interval = end_time[counter_no] - end_time2[counter_no]
        
        if interval < 120:
                end_time[counter_no] = end_time2[counter_no] + 120
        else:
            end_time[counter_no] = end_time[counter_no]
        big_hand = int(end_time[counter_no])//3600
        little_hand = math.ceil(int(end_time[counter_no]) % 3600/60)
        if little_hand >= 60:
            big_hand += 1
            little_hand = little_hand - 60
        if len(str(little_hand)) == 1:
            runway = str(big_hand) + ":0" + str(little_hand)
        else:
            runway = str(big_hand) + ":" + str(little_hand)
        end_time2[counter_no] = end_time[counter_no]
        wait_time = math.ceil((end_time[counter_no] - arrival_time)/60)
        
        data.append([str(line[i+1][0]), "", str(line[i+1][1]), "", "着陸",str(bbb),str(runway),wait_time])
        
        
    else:
        if line[i+1][2] in ["53", "54", "55", "56", "57", "58"]:
            aaa = str("I" + line[i+1][2])
        else:
            aaa = str("D" + line[i+1][2])

        if aaa == "D100":
            print(line[i+1][0], "", line[i+1][1], "", "欠航")
            data.append([str(line[i+1][0]), "", str(line[i+1][1]), "", "欠航"])
        else:
            start_time = arrival_time if arrival_time > end_time[counter_no] else end_time[counter_no]
            end_time[counter_no] = start_time + int(nx.dijkstra_path_length(G, aaa, bbb))/5.55
            interval = end_time[counter_no] - end_time2[counter_no]

            if interval < 120:
                end_time[counter_no] = end_time2[counter_no] + 120
            else:
                end_time[counter_no] = end_time[counter_no]

            wait_time = (end_time[counter_no] - int(nx.dijkstra_path_length(G, aaa, bbb))/5.55 - start_time)/60
            big_hand = int(end_time[counter_no])//3600
            little_hand = math.ceil(int(end_time[counter_no]) % 3600/60) + 3
            if little_hand >= 60:
                big_hand += 1
                little_hand = little_hand - 60
            if len(str(little_hand)) == 1:
                runway = str(big_hand) + ":0" + str(little_hand)
            else:
                runway = str(big_hand) + ":" + str(little_hand)

            end_time2[counter_no] = end_time[counter_no]
            wait_time_heikin += wait_time

            go_test = (arrival_time + (math.ceil(wait_time)*60))
            if len(str(math.ceil(int(go_test) % 3600/60))) == 1:
                go_time = str(go_test//3600) + ":0" + \
                    str(math.ceil(int(go_test) % 3600/60))
            else:
                go_time = str(go_test//3600) + ":" + \
                    str(math.ceil(int(go_test) % 3600/60))

            data.append([str(line[i+1][0]), str(go_time), str(line[i+1][1]),"",
                        str(aaa), str(bbb), runway, str(math.ceil(wait_time))])

            print('{} {} {} {} {} {} {}'
                .format(str(line[i+1][0]), str(go_time), str(line[i+1][1]),"", str(aaa), str(bbb), runway, str(math.ceil(wait_time))))

data.append([])
data.append(["平均遅延時間", wait_time_heikin/(i+1)*60, "[s]"])
df_list = pd.DataFrame(
    data, columns=["定刻", "出発時間", "行先","","ゲート番号", "滑走路", "滑走路到着時間", "遅延"])
df_list.to_csv("and2.csv", index=False, encoding="shift_jis")
print("平均遅延時間", wait_time_heikin/(i+1)*60, "[s]")

7:00 7:00 東京  D10 X 7:06
7:00 7:00 東京  D8 S 7:05
7:00 7:00 大阪  D81 S 7:09
7:00 7:00 東京  D2 X 7:09
7:00 7:00 東京  D4 X 7:11
7:05 7:06 大阪  D6 S 7:11
7:05 7:06 奄美大島  D6 X 7:13
7:10 7:10 沖縄/那覇  D82 S 7:18
7:10 7:10 宮崎  D12 X 7:17
7:10 7:10 宮崎  D82 X 7:22
7:15 7:15 出雲  D81 S 7:23
7:15 7:15 沖縄/那覇  D12 X 7:25
7:20 7:20 新潟  D82 S 7:28
7:25 7:25 小松  D81 X 7:34
7:25 7:25 成田  D3 S 7:30
7:35 7:35 東京  D7 S 7:40
7:40 7:40 名古屋/中部  D5 X 7:46
7:40 7:40 対馬  D82 S 7:48
7:45 7:45 福江  D82 X 7:54
7:50 7:50 松山  D81 S 7:58
7:55 7:55 仙台  D81 X 8:04
7:55 7:55 成田  D1 S 8:00
8:00 8:00 東京  D11 S 8:06
8:00 8:01 沖縄/那覇  D6 X 8:06
8:00 8:00 沖縄/那覇  D4 X 8:08
8:15 8:15 信州松本  D1 S 8:20
8:25 8:25 東京  D8 S 8:30
8:50 8:50 天草  D81 S 8:58
8:55 8:55 名古屋/中部  D5 X 9:01
8:55 8:56 沖縄/那覇  D4 S 9:00
8:55 8:55 仙台  D11 S 9:02
8:55 8:55 東京  D2 X 9:03
9:00 9:00 東京  D9 S 9:05
9:00 9:00 大阪  D11 X 9:06
9:00 9:01 高知  D6 S 9:07
9:05 9:06 名古屋/小牧  D3 S 9:11
9:10 9:10 東京  D7 S 9:15
9:10 9:10 東京  D12 X 9:17
9:10 9:10 バンコク/スワンナプーム  I54 S 9:20
9:15